In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("MAT")
Pkg.add("Printf")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("JLD")
Pkg.add("Plots")

using JuMP
using Gurobi
using MAT
using Printf
using DataFrames
using CSV
using Plots
using JLD

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

In [2]:
# High-level Settings
Zone = "NORTH" # price zone name

# read RTP price
fileln = matopen(string("./RTP_",Zone,"_2019_2021_1h.mat"))
RTP = read(fileln, string("RTP_",Zone,"_2019_2021_1h"))
close(fileln)

# read DAP price
fileln = matopen(string("./DAP_",Zone,"_2019_2021.mat"))
DAP = read(fileln, string("DAP_",Zone,"_2019_2021_Julia"))
close(fileln)

# read DA power decision
fileln = matopen(string("./pDA_",Zone,".mat"))
pDA_real = read(fileln, string("pS_DA"))
close(fileln)
pDA_real = pDA_real[1:8760]
# pDA_real = reshape(pDA_real,(24,365));

# inds = findall(i->(i<0), pDA_real);
# neginds = getindex.(inds, 1);
# inds = findall(i->(i>=0), pDA_real);
# posinds = getindex.(inds, 1);

# pDA_rdis = zeros(1,length(pDA_real));
# pDA_rdis[posinds] .= pDA_real[posinds];
# pDA_rdis = transpose(pDA_rdis);
# pDA_rdis = reshape(pDA_rdis,(24,365));

# pDA_rchr = zeros(1,length(pDA_real));
# pDA_rchr[neginds] .= -pDA_real[neginds];
# pDA_rchr = transpose(pDA_rchr);
# pDA_rchr = reshape(pDA_rchr,(24,365));


# RTP = reshape(RTP,(8760,1));
# # RTP = RTP[1:24]
# DAP = reshape(DAP,(8760,1));
# # DAP = DAP[1:24];

8760-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.05666666666666671
 -0.4988888888888888
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [3]:
inds = findall(i->(i<0), pDA_real);
neginds = getindex.(inds, 1);
inds = findall(i->(i>=0), pDA_real);
posinds = getindex.(inds, 1);

pDA_rdis = zeros(length(pDA_real),1);
pDA_rdis[posinds] .= pDA_real[posinds];
pDA_rdis = reshape(pDA_rdis,(24,365));

pDA_rchr = zeros(length(pDA_real),1);
pDA_rchr[neginds] .= -pDA_real[neginds];
pDA_rchr = reshape(pDA_rchr,(24,365));

pDA_real = reshape(pDA_real,(24,365));

pDA_real

24×365 Matrix{Float64}:
  0.0         0.0         0.45      …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0         0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0         0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0        -0.5         0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0        -0.0555556   0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0         0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0        -0.113333     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0        -0.498889     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0        -0.498889     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0566667   0.0         0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.498889    0.0         0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0         0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0         0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0         0.0         0

In [4]:
## battery setting

E = 1;  # storage energy capacity in MWh
# simulation setting
T = 24; # total time steps
P = .5; # power rating MW
eta = .9; # efficiency
e0 = .5 * E;
ef = e0;
MC = 10; # marginal discharge cost
M = 1; # duaration per step in hour
N_sim = 365; # number of days
d_DAP = DAP[:,1]; # daily DA price
d_RTP = RTP[:,1];
d_pDA = pDA_real[:,1];

# initialize optimization model
model = Model(Gurobi.Optimizer)
set_silent(model) # no outputs

# @variable(model, d_da[1:T], lower_bound = 0) # discharge power in DA
@variable(model, d_rt[1:T], lower_bound = 0) # discharge power in RT
# @variable(model, c_da[1:T], lower_bound = 0) # charge power in DA
@variable(model, c_rt[1:T], lower_bound = 0) # charge power in RT
@variable(model, diff[1:T]) # difference between RT and DA power decision

@variable(model, e[1:T], lower_bound = 0) # total battery energy
@variable(model, R) # market revenue
@variable(model, C) # total degradation cost
@variable(model, u_rt[1:T], Bin) # 1 if charge/buy in RTM

@constraint(model, DiffPower[t=1:T], diff[t] - d_rt[t] + c_rt[t] == -d_pDA[t] ) # actual dis/charge power 

# arbitrage revenue
@constraint(model, ArbRev, M*sum(d_RTP.* diff ) - R == -M*sum(d_DAP.* d_pDA)  )
# @constraint(model, ArbRev, M*sum(RTP.* diff) - R == 0 )

# total degradation cost
@constraint(model, DegCost, M*sum(d_rt * MC) - C == 0 )
# @constraint(model, DegCost, C == M*sum(diff .* MC))

# initial SoC evolution
@constraint(model, SoCInit, e[1] - e0 == M*(c_rt[1]*eta - d_rt[1]/eta) )
# rest SoC evolution
@constraint(model, SoCCont[t = 2:T], e[t] - e[t-1] == M*(c_rt[t]*eta - d_rt[t]/eta) )

# final energy level
@constraint(model, Enelast, e[T] >= ef )
# @constraint(model, Enelast[t = 24:24:T], e[t] >= ef )

# charging / discharging non-conflict condition
@constraint(model, ChRatingTot[t=1:T], c_rt[t] <= P*u_rt[t])
@constraint(model, DchRatingTot[t=1:T], d_rt[t] <= P*(1-u_rt[t]))

# max energy level
@constraint(model, SoCMax[t=1:T], e[t] <= E )

# maximize revenue plus degradation value
@objective(model, Max, R-C);
# @objective(model, Max, R);
print(model);
# ArbRev


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [5]:
# initialize
R_s = zeros(1, N_sim)
P_s = zeros(1, N_sim)
C_s = zeros(1, N_sim)

# C_v = zeros(24, N_sim) # daily charge
C_v_da = zeros(24, N_sim) # daily DA charge
C_v_da = pDA_rchr
C_v_rt = zeros(24, N_sim) # daily RT charge

# D_v = zeros(24, N_sim) # daily discharge
D_v_da = zeros(24, N_sim) # daily DA discharge
D_v_da = pDA_rdis
D_v_rt = zeros(24, N_sim) # daily RT discharge

soc = zeros(24, N_sim) # daily SoC

# step = 0

@time begin
@printf("Optimization starts...\n")

for n = 1:(N_sim)       
    # update prices
    local pda = DAP[:,n]
    local prt = RTP[:,n]
    local dailyDAp = pDA_real[:,n]

    # update prices in constraints
    for t = 1:T
        set_normalized_coefficient(ArbRev, diff[t], M*prt[t])
        set_normalized_rhs(ArbRev, -M*sum(pda .* dailyDAp))
        set_normalized_rhs(DiffPower[t], -dailyDAp[t])            
        # step = step+1 
        
    end

    optimize!(model)
    
    global R_s[n] = value(R)
    global C_s[n] = value(C)
    global P_s[n] = objective_value(model)
    # global P_s[n] = objective_value(model)

    
    # global D_s[n] = sum(value.(d_da) + value.(d_rt))
    # global C_v_da[:,n] = value.(c_da)*M
    global C_v_rt[:,n] = value.(c_rt)*M

    # global D_v_da[:,n] = value.(d_da)*M
    global D_v_rt[:,n] = value.(d_rt)*M

    # global C_v[:,n] = (C_v_da[:,n] .+ value.(c_rt))
    # global D_v[:,n] = (D_v_da[:,n] .+ value.(d_rt))
    
    global soc[:,n] = value.(e)
    termination_status(model)
    # global e0 = value.(e)[end]
        
    @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, Cummulative Cost %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), sum(C_s), termination_status(model))
    # @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), termination_status(model))
 
end
    
end
# SoC = vec(reshape(soc,(8760,1)))
# df2 = DataFrame(SoC = SoC)
# CSV.write("SoC_eta.csv", df2)
# #
# save optimal dispatch to csv

rtp = vec(reshape(RTP,(8760,1)))
dap = vec(reshape(DAP,(8760,1)))

DAcharge = vec(reshape(C_v_da,(8760,1)))
RTcharge = vec(reshape(C_v_rt,(8760,1)))

DAdischarge = vec(reshape(D_v_da,(8760,1)))
RTdischarge = vec(reshape(D_v_rt,(8760,1)))
SoC = vec(reshape(soc,(8760,1)))
df = DataFrame(DAP = dap, RTP = rtp, DA_Charge = DAcharge, RT_Charge = RTcharge, DA_Discharge = DAdischarge, RT_Discharge = RTdischarge, SoC = SoC)
CSV.write("dispatch_0911_dbd.csv", df);

Optimization starts...
Day 1, Cummulative Rev 35, Cummulative Profit 18, Cummulative Cost 18, OptStatus: OPTIMAL 
Day 2, Cummulative Rev 56, Cummulative Profit 30, Cummulative Cost 27, OptStatus: OPTIMAL 
Day 3, Cummulative Rev 80, Cummulative Profit 49, Cummulative Cost 31, OptStatus: OPTIMAL 
Day 4, Cummulative Rev 100, Cummulative Profit 64, Cummulative Cost 36, OptStatus: OPTIMAL 
Day 5, Cummulative Rev 105, Cummulative Profit 65, Cummulative Cost 40, OptStatus: OPTIMAL 
Day 6, Cummulative Rev 126, Cummulative Profit 77, Cummulative Cost 49, OptStatus: OPTIMAL 
Day 7, Cummulative Rev 143, Cummulative Profit 86, Cummulative Cost 57, OptStatus: OPTIMAL 
Day 8, Cummulative Rev 186, Cummulative Profit 117, Cummulative Cost 70, OptStatus: OPTIMAL 
Day 9, Cummulative Rev 217, Cummulative Profit 138, Cummulative Cost 79, OptStatus: OPTIMAL 
Day 10, Cummulative Rev 257, Cummulative Profit 160, Cummulative Cost 97, OptStatus: OPTIMAL 
Day 11, Cummulative Rev 333, Cummulative Profit 214, Cum